In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

ruta_datos=Path("..")/"datos"/"raw"/"train.csv"
df=pd.read_csv(ruta_datos)

In [ ]:
df.head()

### **Fase 1: Diagnóstico y Memoria (La base del rendimiento)**

**1. Concepto: Inspección Profunda (`memory_usage='deep'`)**

* **Teoría:** `df.info()` miente a veces. El parámetro `'deep'` interroga el uso real de RAM de los objetos (strings).
* **Reto:** Carga el archivo `train.csv`. ¿Cuánta memoria RAM consume exactamente el DataFrame inicial? Compara el resultado con y sin el argumento `deep`.


In [3]:
df.memory_usage(deep="True")

Index                       132
id                     96270504
vendor_id              11669152
pickup_datetime       110856944
dropoff_datetime      110856944
passenger_count        11669152
pickup_longitude       11669152
pickup_latitude        11669152
dropoff_longitude      11669152
dropoff_latitude       11669152
store_and_fwd_flag     84601352
trip_duration          11669152
dtype: int64

In [4]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 461.8 MB


**2. Concepto: Downcasting Numérico (`pd.to_numeric`)**

* **Teoría:** `int64` es excesivo para contar pasajeros (máximo 6-9).
* **Reto:** Identifica las columnas `passenger_count` y `vendor_id`. Conviértelas al tipo de entero más pequeño posible (`int8` o `int16`) usando `pd.to_numeric(..., downcast='integer')`. ¿Cuántos MB ahorraste solo con esto?


In [10]:
df['passenger_count']=pd.to_numeric(df['passenger_count'],downcast='integer')
df['vendor_id']=pd.to_numeric(df['vendor_id'],downcast='integer')

In [11]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int8   
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int8   
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(1), int8(2), object(4)
memory usage: 442.4 MB


**3. Concepto: Categoricals (`astype('category')`)**

* **Teoría:** Los strings repetitivos (como 'store_and_fwd_flag') consumen mucha RAM. Categorizarlos guarda índices numéricos.
* **Reto:** Convierte la columna `store_and_fwd_flag` a tipo `category`. Verifica los valores únicos y confirma la reducción de memoria.

---


In [12]:
df['store_and_fwd_flag'].unique()

array(['N', 'Y'], dtype=object)

In [14]:
df['store_and_fwd_flag'] = df['store_and_fwd_flag'].astype('category')


In [15]:
df['store_and_fwd_flag'].cat.categories

Index(['N', 'Y'], dtype='object')

In [16]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   id                  1458644 non-null  object  
 1   vendor_id           1458644 non-null  int8    
 2   pickup_datetime     1458644 non-null  object  
 3   dropoff_datetime    1458644 non-null  object  
 4   passenger_count     1458644 non-null  int8    
 5   pickup_longitude    1458644 non-null  float64 
 6   pickup_latitude     1458644 non-null  float64 
 7   dropoff_longitude   1458644 non-null  float64 
 8   dropoff_latitude    1458644 non-null  float64 
 9   store_and_fwd_flag  1458644 non-null  category
 10  trip_duration       1458644 non-null  int64   
dtypes: category(1), float64(4), int64(1), int8(2), object(3)
memory usage: 363.1 MB


### **Fase 2: Vectorización y Tiempo (Adiós a los bucles)**

**4. Concepto: Datetime Vectorizado (`to_datetime` con formato)**

* **Teoría:** Dejar que Pandas "adivine" el formato es lento. Especificar el `format` acelera la conversión.
* **Reto:** Convierte `pickup_datetime` y `dropoff_datetime` a objetos datetime. Mide el tiempo que tarda especificando `format='%Y-%m-%d %H:%M:%S'` vs no especificarlo.

In [17]:
import time

inicio = time.perf_counter()

df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

fin = time.perf_counter()

print(f"Tiempo sin format: {fin - inicio:.4f} segundos")


Tiempo sin format: 1.6008 segundos


In [24]:
df['pickup_datetime'] = df['pickup_datetime'].astype(str)
df['dropoff_datetime'] = df['dropoff_datetime'].astype(str)

In [22]:
inicio = time.perf_counter()

df['pickup_datetime'] = pd.to_datetime(
    df['pickup_datetime'],
    format='%Y-%m-%d %H:%M:%S'
)

df['dropoff_datetime'] = pd.to_datetime(
    df['dropoff_datetime'],
    format='%Y-%m-%d %H:%M:%S'
)

fin = time.perf_counter()

print(f"Tiempo con format: {fin - inicio:.4f} segundos")


Tiempo con format: 1.6384 segundos


In [23]:
df[['pickup_datetime', 'dropoff_datetime']].dtypes

pickup_datetime     datetime64[ns]
dropoff_datetime    datetime64[ns]
dtype: object

**5. Concepto: Accessors de Fecha (`.dt`)**

* **Teoría:** Extraer partes de una fecha es instantáneo con `.dt`, sin necesidad de lambdas.
* **Reto:** Crea dos nuevas columnas: `hour_of_day` y `day_of_week` usando el accessor `.dt`. ¿Cuál es la hora pico (la hora con más viajes iniciados) en NYC?

In [29]:
df['pickup_datetime'].dtype

dtype('O')

In [30]:
df['pickup_datetime'] = pd.to_datetime(
    df['pickup_datetime'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

In [31]:
df['pickup_datetime'].dtype


dtype('<M8[ns]')

In [32]:
df['hour_of_day'] = df['pickup_datetime'].dt.hour
df['day_of_week'] = df['pickup_datetime'].dt.dayofweek

In [33]:
df['day_of_week_name'] = df['pickup_datetime'].dt.day_name()

In [34]:
viajes_por_hora = df['hour_of_day'].value_counts().sort_index()


In [36]:
hora_pico = viajes_por_hora.idxmax()
total_viajes = viajes_por_hora.max()

hora_pico, total_viajes

(np.int32(18), np.int64(90600))

La mayor cantidad de viajes en NYC se inicia a las 6:00 p. m.Número de viajes iniciados: 90 600

**6. Concepto: Operaciones Vectorizadas Complejas (NumPy)**

* **Teoría:** Usar `np.where` es infinitamente más rápido que `apply` con `if/else`.
* **Reto:** Crea una columna `trip_type`. Si la `trip_duration` es mayor a 3600 segundos (1 hora), etiquétalo como "Largo", de lo contrario "Corto". Hazlo usando `np.where`.

In [37]:
import numpy as np

df['trip_type'] = np.where(
    df['trip_duration'] > 3600,
    'Largo',
    'Corto'
)

In [38]:
df['trip_type'].value_counts()

trip_type
Corto    1446327
Largo      12317
Name: count, dtype: int64

Corto (1,446,327): número de viajes cuya duración fue menor o igual a 1 hora (≤ 3600 segundos).

Largo (12,317): número de viajes cuya duración fue mayor a 1 hora (> 3600 segundos).

**7. Concepto: Matemáticas Geoespaciales (La prueba de fuego)**

* **Teoría:** Calcular distancias con trigonometría (Haversine) usando columnas enteras en lugar de filas.
* **Reto:** Implementa la fórmula de Haversine usando funciones de NumPy (`np.sin`, `np.cos`, `np.sqrt`) para calcular la distancia en Km entre (pickup_lat, pickup_lon) y (dropoff_lat, dropoff_lon). Crea la columna `distance_km`.

In [42]:
import numpy as np

R = 6371  # Radio de la Tierra en km

# Convertir grados a radianes (vectorizado)
lat1 = np.radians(df['pickup_latitude'])
lon1 = np.radians(df['pickup_longitude'])
lat2 = np.radians(df['dropoff_latitude'])
lon2 = np.radians(df['dropoff_longitude'])

# Diferencias
dlat = lat2 - lat1
dlon = lon2 - lon1

# Fórmula de Haversine
a = (
    np.sin(dlat / 2) ** 2
    + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
)

c = 2 * np.arcsin(np.sqrt(a))

# Distancia final
df['distance_km'] = R * c


In [44]:
df['distance_km'].describe()

count    1.458644e+06
mean     3.440864e+00
std      4.296538e+00
min      0.000000e+00
25%      1.231837e+00
50%      2.093717e+00
75%      3.875337e+00
max      1.240909e+03
Name: distance_km, dtype: float64

---

### **Fase 3: Method Chaining y Transformación (Código Elegante)**

**8. Concepto: `.query()**`

* **Teoría:** Filtrar con strings es más legible y, en grandes volúmenes, a veces más eficiente que el boolean indexing tradicional.
* **Reto:** Filtra el dataset para quedarte solo con viajes que: tengan más de 1 pasajero, la distancia sea mayor a 0 y menor a 100 km. Todo en una sola línea de `.query()`.


In [45]:
df_filtrado = df.query(
    "passenger_count > 1 and distance_km > 0 and distance_km < 100"
)

In [46]:
df_filtrado.describe()[['passenger_count', 'distance_km']]

,passenger_count,distance_km
count,423345.000000,423345.000000
mean,3.280131,3.586580
min,2.000000,0.000424
25%,2.000000,1.268909
50%,3.000000,2.152577
75%,5.000000,4.033796
max,8.000000,83.088767
std,1.496761,4.126718



**9. Concepto: `.assign()**`

* **Teoría:** Permite crear columnas al vuelo dentro de una cadena sin romper el flujo.
* **Reto:** En un solo bloque encadenado: toma el df filtrado, crea una columna `velocidad_kph` (distancia / (duración/3600)) y ordena los valores por velocidad descendente.


In [48]:
df_velocidad = (
    df_filtrado
        .assign(
            velocidad_kph=lambda x: x['distance_km'] / (x['trip_duration'] / 3600)
        )
        .sort_values('velocidad_kph', ascending=False)
)

In [49]:
df_velocidad[['distance_km', 'trip_duration', 'velocidad_kph']].head()


,distance_km,trip_duration,velocidad_kph
1001028,0.783347,2,1410.024743
1107,0.703065,2,1265.516683
1322903,1.029739,4,926.765395
1200843,0.760976,3,913.171260
693299,19.619959,121,583.734313



**10. Concepto: `.clip()` (Manejo de Outliers)**

* **Teoría:** En lugar de borrar filas extremas, a veces queremos "toparlas" a un límite lógico.
* **Reto:** Hay viajes con duraciones absurdas. Usa `.clip()` para limitar la columna `trip_duration` a un máximo de 7200 segundos (2 horas) y un mínimo de 60 segundos. Reemplaza la columna original.

In [50]:
df['trip_duration'] = df['trip_duration'].clip(lower=60, upper=7200)


In [51]:
df['trip_duration'].describe()

count    1.458644e+06
mean     8.460965e+02
std      6.997172e+02
min      6.000000e+01
25%      3.970000e+02
50%      6.620000e+02
75%      1.075000e+03
max      7.200000e+03
Name: trip_duration, dtype: float64


**11. Concepto: `.pipe()` (Funciones personalizadas)**

* **Teoría:** Inserta una función defina por el usuario en la cadena.
* **Reto:** Define una función `def etiqueta_trafico(df):` que reciba el dataframe y devuelva solo los viajes ocurridos en "Hora Punta" (ej: 7-9am y 5-7pm). Aplícala usando `.pipe()` al final de tu cadena anterior.

---


In [53]:
def etiqueta_trafico(df):
    return df.query(
        "(hour_of_day >= 7 and hour_of_day <= 9) or "
        "(hour_of_day >= 17 and hour_of_day <= 19)"
    )

In [54]:
df_hora_punta = (
    df_filtrado
        .assign(
            velocidad_kph=lambda x: x['distance_km'] / (x['trip_duration'] / 3600)
        )
        .sort_values('velocidad_kph', ascending=False)
        .pipe(etiqueta_trafico)
)

In [55]:
df_hora_punta['hour_of_day'].value_counts().sort_index()

hour_of_day
7     13040
8     16554
9     16963
17    23050
18    26765
19    26985
Name: count, dtype: int64

### **Fase 4: Agregaciones Avanzadas y Ventanas**

**12. Concepto: `groupby()` con Named Aggregation**

* **Teoría:** La sintaxis moderna permite renombrar columnas al momento de agregar.
* **Reto:** Agrupa por `vendor_id` y calcula: promedio de duración (llámalo `mean_duration`) y total de distancia (llámalo `total_km`). Todo en un solo paso.

In [56]:
resumen_vendor = (
    df
        .groupby('vendor_id')
        .agg(
            mean_duration=('trip_duration', 'mean'),
            total_km=('distance_km', 'sum')
        )
)

In [58]:
resumen_vendor.reset_index()

,vendor_id,mean_duration,total_km
0,1,830.988072,2.308734e+06
1,2,859.230843,2.710261e+06


**13. Concepto: `.transform()` (La magia oculta)**

* **Teoría:** A diferencia de `agg` (que reduce filas), `transform` mantiene el mismo número de filas original. Ideal para comparar "fila vs promedio de su grupo".
* **Reto:** Crea una columna `avg_duration_by_hour`. Cada fila debe tener la duración promedio de *todos los viajes que ocurrieron en esa misma hora*. (Usa `groupby('hour')['duration'].transform('mean')`).


In [59]:
df['avg_duration_by_hour'] = (
    df
        .groupby('hour_of_day')['trip_duration']
        .transform('mean')
)


**14. Concepto: Filtrado por Grupos (`filter`)**

* **Teoría:** Filtrar grupos enteros basados en una propiedad del grupo, no de la fila.
* **Reto:** Queremos analizar solo las horas del día que son "activas". Filtra el dataframe para mantener solo los datos de las horas (`hour_of_day`) que tengan más de 10,000 viajes en total.


In [65]:
df_activo = (
    df
    .groupby('hour_of_day')
    .filter(lambda x: len(x) > 10_000)
)

In [66]:
df_activo['hour_of_day'].value_counts()

hour_of_day
18    90600
19    90308
21    84185
20    84072
22    80492
17    76483
14    74292
12    71873
15    71811
13    71473
23    69785
11    68476
9     67663
8     67053
10    65437
16    64313
7     55600
0     53248
1     38571
6     33248
2     27972
3     20895
4     15792
5     15002
Name: count, dtype: int64


**15. Concepto: `cut` y `qcut` (Binning)**

* **Teoría:** Convertir variables continuas en discretas (rangos).
* **Reto:** Usa `pd.qcut` para dividir la columna `distance_km` en 4 cuartiles: "Muy Corto", "Corto", "Largo", "Muy Largo". ¿Cuál es la duración promedio de los viajes "Muy Largos"?

---

In [67]:
df['distance_category'] = pd.qcut(
    df['distance_km'],
    q=4,
    labels=['Muy Corto', 'Corto', 'Largo', 'Muy Largo']
)

In [70]:
mean_duration_muy_largo = (
    df
    .loc[df['distance_category'] == 'Muy Largo', 'trip_duration']
    .mean()
)

mean_duration_muy_largo

np.float64(1568.2715508376273)

In [71]:
df['distance_category'].value_counts()

distance_category
Muy Corto    364661
Corto        364661
Largo        364661
Muy Largo    364661
Name: count, dtype: int64

### **Fase 5: Series Temporales y Visualización (Storytelling)**

**16. Concepto: `resample()**`

* **Teoría:** Como un groupby, pero especializado en índices de tiempo (frecuencias: Hora, Día, Mes).
* **Reto:** Establece `pickup_datetime` como índice. Resamplea los datos por 'D' (Día) y calcula el conteo total de viajes por día. ¿Qué día de la semana tiene menos tráfico históricamente?


In [73]:
df = df.set_index('pickup_datetime')

In [74]:
daily_trips = df.resample('D').size()

In [75]:
daily_trips_by_weekday = (
    daily_trips
    .groupby(daily_trips.index.day_name())
    .mean()
)

In [76]:
daily_trips_by_weekday.sort_values()

pickup_datetime
Monday       7208.384615
Sunday       7514.076923
Tuesday      7798.038462
Wednesday    8082.153846
Thursday     8406.692308
Saturday     8494.923077
Friday       8597.423077
dtype: float64

**17. Concepto: Rolling Windows (`rolling()`)**

* **Teoría:** Suavizar líneas temporales ruidosas.
* **Reto:** Sobre la serie temporal diaria anterior, calcula una media móvil de 7 días (`rolling(7).mean()`). Grafica la serie original vs la suavizada.

**18. Concepto: Visualización de Relaciones (Scatter con alpha)**

* **Teoría:** Con tantos datos, un scatter plot normal es una mancha sólida. El parámetro `alpha` es vital.
* **Reto:** Haz un Scatter Plot de `distance_km` vs `trip_duration`. Usa `alpha=0.1` para ver la densidad de puntos. ¿Ves líneas horizontales extrañas? (Son tarifas fijas o errores).

**19. Concepto: Heatmaps de Correlación**

* **Teoría:** Entender qué variables se mueven juntas.
* **Reto:** Crea una matriz de correlación (`df.corr()`) de las variables numéricas y visualízala con un Heatmap de Seaborn. ¿La cantidad de pasajeros influye en la duración del viaje?

**20. Concepto: Exportación Eficiente (`to_parquet`)**

* **Teoría:** CSV es lento y pesado. Parquet mantiene los tipos de datos y comprime.
* **Reto:** Guarda tu DataFrame final limpio como `clean_taxi.parquet`. Compara el tamaño del archivo resultante contra el CSV original.
 